# Results

## Table of Contents
* [1)Reproducing Table 1 in in Longstaff-Schwartz](#first-bullet)
* [2) *American Put Prices* Applying Least-Square Monte Carlo with Our Own Parameter Values](#second-bullet)
    - [2a) Black-Scholes](#seconda-bullet)
    - [2b) Jump Merton](#secondb-bullet)
        - [2b extra) Jump Merton (BAD TRUNCATION)](#secondba-bullet)
    - [2c) Constant Elasticity of Variance (CEV)](#secondc-bullet)

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import os

# local
from DC import DynamicChebyshev
from LSM import MonteCarloOptionPricing
from FD import AmericanPutFiniteDifference

## 1) *American Put Prices* Applying Least-Square Monte Carlo with Longstaff-Schwartz Parameters <a class="anchor" id="first-bullet"></a>
The simulation is based on 100,000 (50,000 plus 50,000 antithetic) paths for the stock price.

In [13]:
# Parameter inputs
r = 0.06  # risk-free rate
K = 40.   # strike price
λ = 0.4 # intensity rate of the Poisson process
dim = 100_000  # number of simulation paths
n = 50  # number of exercise rights per year
seed = 15_001  # random seed for reproducibility
use_AV = True  # antithetic variates
poly_degree = 3  # polynomial degree for LSM regression

In [14]:
# Varying option parameters
S0_values = [36, 38, 40, 42, 44]
σ_values = [0.20, 0.40]
T_values = [1, 2]

**Black-Scholes (*Akin to Table 1 in Longstaff-Schwartz*)**:

In [15]:
# Store results
Option_Pricing_Results_BS = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_BS = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Calculate European Put using Black-Scholes
            BS_Price = MC_Option_Pricing_BS.BS_option_value('put')
            
            # 2) Compute American Put using LSM
            MC_Option_Pricing_BS.GeometricBrownianMotion_vec() # simulate GBM paths
            American_put_price = MC_Option_Pricing_BS.american_option_LSM(poly_degree, otype='put') # price option
            
            # 3) Calculate premium
            Option_Premium_LSM = American_put_price[0] - BS_Price 
            
            # 4) Compute American Put using Implicit Finite Difference
            Implicit_Finite_Difference = AmericanPutFiniteDifference(K, r, M = 1_000)
            FD = Implicit_Finite_Difference.implicit_FD(S0, σ, T, T*40_000)
            
            # 5) Calculate premium
            Option_Premium_FD = FD - BS_Price 
            
            # 6) diff in premium
            Diff_prem = Option_Premium_FD - Option_Premium_LSM
            
            # 7) Store the results
            Option_Pricing_Results_BS.append((S0, σ, T, FD, BS_Price, Option_Premium_FD, American_put_price[0],American_put_price[3], Option_Premium_LSM, Diff_prem))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, F.D.: {FD:.3f}, EUR Put: {BS_Price:.3f}, Premium: {Option_Premium_FD:.3f}  LSM: {American_put_price[0]:.3f}, s.e. {American_put_price[3]:.3f}, Premium: {Option_Premium_LSM:.3f}, Diff. in Prem:{Diff_prem:.3f}')

S0/Sigma/T: 36/0.2/1, F.D.: 4.487, EUR Put: 3.844, Premium: 0.642  LSM: 4.472, s.e. 0.009, Premium: 0.628, Diff. in Prem:0.015
S0/Sigma/T: 36/0.2/2, F.D.: 4.848, EUR Put: 3.763, Premium: 1.085  LSM: 4.826, s.e. 0.011, Premium: 1.063, Diff. in Prem:0.022
S0/Sigma/T: 36/0.4/1, F.D.: 7.106, EUR Put: 6.711, Premium: 0.395  LSM: 7.075, s.e. 0.019, Premium: 0.364, Diff. in Prem:0.031
S0/Sigma/T: 36/0.4/2, F.D.: 8.441, EUR Put: 7.700, Premium: 0.741  LSM: 8.459, s.e. 0.023, Premium: 0.759, Diff. in Prem:-0.018
S0/Sigma/T: 38/0.2/1, F.D.: 3.257, EUR Put: 2.852, Premium: 0.405  LSM: 3.240, s.e. 0.009, Premium: 0.388, Diff. in Prem:0.017
S0/Sigma/T: 38/0.2/2, F.D.: 3.751, EUR Put: 2.991, Premium: 0.761  LSM: 3.727, s.e. 0.011, Premium: 0.736, Diff. in Prem:0.025
S0/Sigma/T: 38/0.4/1, F.D.: 6.153, EUR Put: 5.834, Premium: 0.319  LSM: 6.122, s.e. 0.019, Premium: 0.288, Diff. in Prem:0.031
S0/Sigma/T: 38/0.4/2, F.D.: 7.619, EUR Put: 6.979, Premium: 0.640  LSM: 7.629, s.e. 0.022, Premium: 0.651, Dif

## 2) *American Put Prices* Applying Least-Square Monte Carlo with Our Own Parameter Values <a class="anchor" id="second-bullet"></a>
The simulation is based on 100,000 (50,000 plus 50,000 antithetic) paths for the stock price.

In [2]:
# Parameter inputs
r = 0.03  # risk-free rate
K = 100.   # strike price
λ = 0.4 # intensity rate of the Poisson process
dim = 100_000  # number of simulation paths
n = 252  # number of exercise rights per year
n_chebyshev_pol = 100 # degree of chebyshev polynomials
seed = 15_001  # random seed for reproducibility
use_AV = True  # antithetic variates
poly_degree = 3  # polynomial degree for LSM regression

In [3]:
# Option parameters
S0_values = [90, 95, 100, 105, 110]
σ_values = [0.20, 0.40]
T_values = [1, 2]

## 2a) Black-Scholes <a class="anchor" id="seconda-bullet"></a>

In [6]:
# Store results
Results_BS = []

# Store results
Option_Pricing_Results_BS = []

# Iterate over each combination of S, sigma, and T
for S0 in S0_values:
    for σ in σ_values:
        for T in T_values:
            # Create instance from class
            MC_Option_Pricing_BS = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)
            
            # 1) Calculate European Put using Black-Scholes
            BS_Price = MC_Option_Pricing_BS.BS_option_value('put')
            
            # 2) Compute American Put using LSM
            MC_Option_Pricing_BS.GeometricBrownianMotion_vec() # simulate GBM paths
            American_put_price = MC_Option_Pricing_BS.american_option_LSM(poly_degree, otype='put') # price option
            
            # 3) Calculate premium
            Option_Premium = American_put_price[0] - BS_Price 
            
            # 4) Compute American Put using Implicit Finite Difference
            Implicit_Finite_Difference = AmericanPutFiniteDifference(K, r, M = 1_000)
            FD = Implicit_Finite_Difference.implicit_FD(S0, σ, T, 40_000)
            
            # 5) Compute American Put using Dynamic Chebyshev 
            DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, 32, n_chebyshev_pol, seed, use_AV)
            
            # a) offline phase
            domain = DC_option_pricing.calculate_truncated_domain_GBM()
            xknots = DC_option_pricing.calculate_nodal_points(domain)
            BS = DC_option_pricing.generate_GBM_path(xknots)
            Γ = DC_option_pricing.compute_generalized_moments(domain, xknots)

            # b) online phase
            DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots, Γ)
            
            # 6) Store the results
            Results_BS.append((S0, σ, T, BS_Price, American_put_price[0], Option_Premium, FD, American_put_price[3], DC[0], DC[1]))
            
            # Print
            print(f'S0/Sigma/T: {S0}/{σ}/{T}, EUR Put: {BS_Price:.4f}, LSM: {American_put_price[0]:.4f}, Premium: {Option_Premium:.4f}, F.D.: {FD:.4f}, s.e.: {American_put_price[3]:.4f}, D.C.: {DC[0]:.4f}, s.e.: {DC[1]:.4f}')

S0/Sigma/T: 90/0.2/1, EUR Put: 11.4925, LSM: 12.1566, Premium: 0.6641, F.D.: 12.1630, s.e.: 0.0274, D.C.: 12.2697, s.e.: 0.0629
S0/Sigma/T: 90/0.2/2, EUR Put: 12.5767, LSM: 13.7671, Premium: 1.1904, F.D.: 13.7741, s.e.: 0.0340, D.C.: 13.9227, s.e.: 0.0770
S0/Sigma/T: 90/0.4/1, EUR Put: 18.6061, LSM: 19.0255, Premium: 0.4194, F.D.: 19.0261, s.e.: 0.0507, D.C.: 19.2287, s.e.: 0.0955
S0/Sigma/T: 90/0.4/2, EUR Put: 22.6607, LSM: 23.5532, Premium: 0.8925, F.D.: 23.3660, s.e.: 0.0620, D.C.: 23.7807, s.e.: 0.1117
S0/Sigma/T: 95/0.2/1, EUR Put: 8.7142, LSM: 9.1444, Premium: 0.4302, F.D.: 9.1538, s.e.: 0.0265, D.C.: 9.2923, s.e.: 0.0592
S0/Sigma/T: 95/0.2/2, EUR Put: 10.2309, LSM: 11.0850, Premium: 0.8541, F.D.: 11.1041, s.e.: 0.0332, D.C.: 11.2800, s.e.: 0.0740
S0/Sigma/T: 95/0.4/1, EUR Put: 16.2649, LSM: 16.5653, Premium: 0.3004, F.D.: 16.6079, s.e.: 0.0493, D.C.: 16.8376, s.e.: 0.0934
S0/Sigma/T: 95/0.4/2, EUR Put: 20.6557, LSM: 21.3921, Premium: 0.7364, F.D.: 21.2829, s.e.: 0.0611, D.C.: 21

## 2b) Jump Merton <a class="anchor" id="secondb-bullet"></a>

In [4]:
λ = 0.4 # intensity rate of the Poisson process
α = -0.5 # Mean of log-normal jump size
β = 0.4 # Volatility of log-normal jump size
σ = 0.25 # diffusion parameter

In [8]:
# Store results
Results_JM = []

# Iterate over each combination of S and T
for S0 in S0_values:
    for T in T_values:
        # Create instance from class
        MC_Option_Pricing_JM = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)

        # 1) Calculate European Put using Jump Merton semi-closed solution
        JumpMerton_price = MC_Option_Pricing_JM.merton_jump_option_value(α, β)

        # 2) Compute American Put using LSM
        MC_Option_Pricing_JM.MertonJumpDiffusion_vec(α, β) # simulate jump paths
        American_put_price = MC_Option_Pricing_JM.american_option_LSM(poly_degree, otype='put') # price option

        # 3) Calculate premium
        Option_Premium = American_put_price[0] - JumpMerton_price 

        # 4) Compute American Put using Dynamic Chebyshev 
        DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, n, n_chebyshev_pol, seed, use_AV)
        # a) offline phase
        domain = DC_option_pricing.calculate_truncated_domain_JumpMerton(α, β)
        xknots = DC_option_pricing.calculate_nodal_points(domain)
        JM = DC_option_pricing.generate_Jump_path(xknots, α, β)
        Γ = DC_option_pricing.compute_generalized_moments(domain,xknots)

        # b) online phase
        DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots,Γ)

        # 5) Store the results
        Results_JM.append((S0, T, JumpMerton_price, American_put_price[0], Option_Premium, DC[0]))

        # Print
        print(f'S0/Sigma/T: {S0}/{T}, EUR Put: {JumpMerton_price:.4f}, LSM: {American_put_price[0]:.4f}, D.C.: {DC[0]:.4f}')

S0/Sigma/T: 90/1, EUR Put: 17.4123, LSM: 17.7160, D.C.: 20.4271
S0/Sigma/T: 90/2, EUR Put: 22.3260, LSM: 23.3259, D.C.: 26.0414
S0/Sigma/T: 95/1, EUR Put: 15.3938, LSM: 15.6155, D.C.: 18.3506
S0/Sigma/T: 95/2, EUR Put: 20.6251, LSM: 21.5210, D.C.: 24.2490
S0/Sigma/T: 100/1, EUR Put: 13.6913, LSM: 13.8622, D.C.: 16.5189
S0/Sigma/T: 100/2, EUR Put: 19.1072, LSM: 19.8982, D.C.: 22.6149
S0/Sigma/T: 105/1, EUR Put: 12.2571, LSM: 12.3934, D.C.: 14.9042
S0/Sigma/T: 105/2, EUR Put: 17.7487, LSM: 18.4351, D.C.: 21.1430
S0/Sigma/T: 110/1, EUR Put: 11.0457, LSM: 11.1449, D.C.: 13.4796
S0/Sigma/T: 110/2, EUR Put: 16.5288, LSM: 17.1719, D.C.: 19.8019


### 2b extra) Jump Merton (BAD TRUNCATION) <a class="anchor" id="secondba-bullet"></a>

In [9]:
# Store results
Results_JM = []

# Iterate over each combination of S and T
for S0 in S0_values:
    for T in T_values:
        # Create instance from class
        MC_Option_Pricing_JM = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)

        # 1) Calculate European Put using Jump Merton semi-closed solution
        JumpMerton_price = MC_Option_Pricing_JM.merton_jump_option_value(α, β)

        # 2) Compute American Put using LSM
        MC_Option_Pricing_JM.MertonJumpDiffusion_vec(α, β) # simulate jump paths
        American_put_price = MC_Option_Pricing_JM.american_option_LSM(poly_degree, otype='put') # price option

        # 3) Calculate premium
        Option_Premium = American_put_price[0] - JumpMerton_price 

        # 4) Compute American Put using Dynamic Chebyshev 
        DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, n, n_chebyshev_pol, seed, use_AV)
        # a) offline phase
        domain = DC_option_pricing.calculate_truncated_domain_GBM()
        xknots = DC_option_pricing.calculate_nodal_points(domain)
        JM = DC_option_pricing.generate_Jump_path(xknots, α, β)
        Γ = DC_option_pricing.compute_generalized_moments(domain,xknots)

        # b) online phase
        DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots,Γ)

        # 5) Store the results
        Results_JM.append((S0, T, JumpMerton_price, American_put_price[0], Option_Premium, DC[0]))

        # Print
        print(f'S0/Sigma/T: {S0}/{T}, EUR Put: {JumpMerton_price:.4f}, LSM: {American_put_price[0]:.4f}, D.C.: {DC[0]:.4f}')

S0/Sigma/T: 90/1, EUR Put: 17.4123, LSM: 17.7160, D.C.: 15.6394
S0/Sigma/T: 90/2, EUR Put: 22.3260, LSM: 23.3259, D.C.: 22.3241
S0/Sigma/T: 95/1, EUR Put: 15.3938, LSM: 15.6155, D.C.: 13.2309
S0/Sigma/T: 95/2, EUR Put: 20.6251, LSM: 21.5210, D.C.: 20.3239
S0/Sigma/T: 100/1, EUR Put: 13.6913, LSM: 13.8622, D.C.: 11.1931
S0/Sigma/T: 100/2, EUR Put: 19.1072, LSM: 19.8982, D.C.: 18.5193
S0/Sigma/T: 105/1, EUR Put: 12.2571, LSM: 12.3934, D.C.: 9.4750
S0/Sigma/T: 105/2, EUR Put: 17.7487, LSM: 18.4351, D.C.: 16.8955
S0/Sigma/T: 110/1, EUR Put: 11.0457, LSM: 11.1449, D.C.: 8.0276
S0/Sigma/T: 110/2, EUR Put: 16.5288, LSM: 17.1719, D.C.: 15.4205


## 2c) Constant Elasticity of Variance (CEV) <a class="anchor" id="secondc-bullet"></a>

In [5]:
γ = 1.5 # parameter governing elasticity with respect to price, if set = 2 then we obtain BS prices

In [11]:
# Store results
Results_CEV = []

# Iterate over each combination of S and T
for S0 in S0_values:
    for T in T_values:
        # Create instance from class
        MC_Option_Pricing_CEV = MonteCarloOptionPricing(r, S0, K, T, σ, λ, dim, n, seed, use_AV)

        # 1) Compute American Put using LSM
        MC_Option_Pricing_CEV.CEV(γ) # simulate CEV paths
        American_put_price = MC_Option_Pricing_CEV.american_option_LSM(poly_degree, otype='put') # price option

        # 2) Compute American Put using Dynamic Chebyshev
        DC_option_pricing = DynamicChebyshev(r, S0, K, T, σ, λ, dim, 32, n_chebyshev_pol, seed, use_AV)
        # a) offline phase
        domain = DC_option_pricing.calculate_truncated_domain_GBM()
        xknots = DC_option_pricing.calculate_nodal_points(domain)
        CEV = DC_option_pricing.generate_CEV_path(xknots, γ)
        Γ = DC_option_pricing.compute_generalized_moments(domain, xknots)

        # b) online phase
        DC = DC_option_pricing.price_option_with_dynamic_chebyshev(xknots, Γ)

        # 3) Store the results
        Results_CEV.append((S0, σ, T, American_put_price[0], DC[0]))

        # Print
        print(f'S0/Sigma/T: {S0}/{T}, LSM: {American_put_price[0]:.4f}, D.C.: {DC[0]:.4f}')

S0/Sigma/T: 90/1, LSM: 9.9881, D.C.: 11.4798
S0/Sigma/T: 90/2, LSM: 9.9762, D.C.: 12.7280
S0/Sigma/T: 95/1, LSM: 5.1661, D.C.: 8.3407
S0/Sigma/T: 95/2, LSM: 5.3761, D.C.: 9.9463
S0/Sigma/T: 100/1, LSM: 2.1379, D.C.: 5.8836
S0/Sigma/T: 100/2, LSM: 2.6115, D.C.: 7.6823
S0/Sigma/T: 105/1, LSM: 0.7258, D.C.: 4.0348
S0/Sigma/T: 105/2, LSM: 1.1687, D.C.: 5.8856
S0/Sigma/T: 110/1, LSM: 0.2001, D.C.: 2.6916
S0/Sigma/T: 110/2, LSM: 0.4747, D.C.: 4.4581
